In [1]:
import tensorflow as tf
import os
import argparse
from tensorflow.python.keras.callbacks import Callback



class MyFashionMnist(object):
  def train(self):
    
    # 입력 값을 받게 추가합니다.
    parser = argparse.ArgumentParser()
    parser.add_argument('--learning_rate', required=False, type=float, default=0.001)
    parser.add_argument('--dropout_rate', required=False, type=float, default=0.2)
    # epoch 5 ~ 15
    parser.add_argument('--epoch', required=False, type=int, default=5)    
    # relu, sigmoid, softmax, tanh
    parser.add_argument('--act', required=False, type=str, default='relu')        
    # layer 1 ~ 5
    parser.add_argument('--layer', required=False, type=int, default=1)        
    
    
    args = parser.parse_args()    
    
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0

    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Flatten(input_shape=(28, 28)))
    
    for i in range(int(args.layer)):    
        model.add(tf.keras.layers.Dense(128, activation=args.act))
        model.add(tf.keras.layers.Dropout(args.dropout_rate))
        
    model.add(tf.keras.layers.Dense(10, activation='softmax'))
    model.summary()
    
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=args.learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['acc'])

    model.fit(x_train, y_train,
              verbose=0,
              validation_data=(x_test, y_test),
              epochs=args.epoch,
              callbacks=[KatibMetricLog()])

    model.evaluate(x_test,  y_test, verbose=0)

class KatibMetricLog(Callback):
    def on_batch_end(self, batch, logs={}):
        print("batch=" + str(batch),
              "accuracy=" + str(logs.get('acc')),
              "loss=" + str(logs.get('loss')))
    def on_epoch_begin(self, epoch, logs={}):
        print("epoch " + str(epoch) + ":")
    
    def on_epoch_end(self, epoch, logs={}):
        print("Validation-accuracy=" + str(logs.get('val_acc')),
              "Validation-loss=" + str(logs.get('val_loss')))
        return

if __name__ == '__main__':
    if os.getenv('FAIRING_RUNTIME', None) is None:
        from kubeflow import fairing
        from kubeflow.fairing.kubernetes import utils as k8s_utils

        DOCKER_REGISTRY = 'kubeflow-registry.default.svc.cluster.local:30000'
        fairing.config.set_builder(
            'append',
            image_name='fairing-job',
            base_image='brightfly/kubeflow-jupyter-lab:tf2.0-cpu',
            registry=DOCKER_REGISTRY, 
            push=True)
        # cpu 2, memory 5GiB
        fairing.config.set_deployer('job',
                                    namespace='dudaji',
                                    pod_spec_mutators=[
                                        k8s_utils.get_resource_mutator(cpu=1,
                                                                       memory=2)]
         
                                   )
        fairing.config.run()
    else:
        remote_train = MyFashionMnist()
        remote_train.train()

[I 200222 04:06:30 config:123] Using preprocessor: <kubeflow.fairing.preprocessors.converted_notebook.ConvertNotebookPreprocessor object at 0x7fe087472898>
[I 200222 04:06:30 config:125] Using builder: <kubeflow.fairing.builders.append.append.AppendBuilder object at 0x7fe035db02e8>
[I 200222 04:06:30 config:127] Using deployer: <kubeflow.fairing.builders.append.append.AppendBuilder object at 0x7fe035db02e8>
[W 200222 04:06:30 append:50] Building image using Append builder...
[I 200222 04:06:30 base:105] Creating docker context: /tmp/fairing_context_yah1yor_
[I 200222 04:06:30 converted_notebook:127] Converting fashion-mnist-katib.ipynb to fashion-mnist-katib.py
[I 200222 04:06:30 docker_creds_:234] Loading Docker credentials for repository 'brightfly/kubeflow-jupyter-lab:tf2.0-cpu'
[W 200222 04:06:32 append:54] Image successfully built in 2.3115801259991713s.
[W 200222 04:06:32 append:94] Pushing image kubeflow-registry.default.svc.cluster.local:30000/fairing-job:506FE0D9...
[I 200222 

4423680/4422102 [==============================] - 1s 0us/step
2020-02-22 04:08:04.219654: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2020-02-22 04:08:04.226354: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 3593190000 Hz
2020-02-22 04:08:04.226973: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x4fddb10 executing computations on platform Host. Devices:
2020-02-22 04:08:04.227013: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): Host, Default Version
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #
flatten (Flatten)            (None, 784)               0
_________________________________________________________________
dense (Dense)                (None, 128)               100480
___________________________________

KeyboardInterrupt: 